<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/Case_Study/Relevant_Job__Title_Pictures_v05_KMeans_new_feat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Describe...

----------
Author: Raiana Roland Seixas
09/07/2022

# Data load & Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.cluster import KMeans

In [3]:
!git clone https://github.com/RaianaSeixas/NLT
%cd NLT

fatal: destination path 'NLT' already exists and is not an empty directory.
/content/NLT


In [4]:
df=pd.read_csv('occupation_pictures.csv')
df

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Picture URL,Manual Label
0,3D Modeler,Our Top 19 3D Modeling Software Picks ...,1261,0,0,7,276.763672,1917,2301,4411017,2,https://format-magazine-production-res.cloudin...,NaN
1,3D Modeler,7 Tips for Beginner 3D Modelers ...,1261,1,0,3,44.343750,340,747,253980,5,http://static1.squarespace.com/static/5320f582...,NaN
2,3D Modeler,How to Become an Expert 3D Modeler ...,1261,2,0,2,43.088867,380,660,250800,8,https://www.gamedesigning.org/wp-content/uploa...,NaN
3,3D Modeler,What is 3D Modeling & What's It Used For?,1261,3,0,2,78.623047,330,750,247500,1,https://cdn.conceptartempire.com/images/04/522...,NaN
4,3D Modeler,Shoofping 3D Modeler ...,1261,4,0,1,97.517578,1075,1909,2052175,3,http://shoofping.com/wp-content/uploads/2017/1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,Wind Turbine Engineer,Wind turbine technician fastest-growing ...,61,4,0,0,89.249023,720,1280,921600,0,https://i.ytimg.com/vi/lUf2mOYp1d4/maxresdefau...,good
654,Wind Turbine Engineer,How to Become a Wind Energy Engineer ...,61,5,0,0,37.949219,200,300,60000,0,https://www.environmentalscience.org/wp-conten...,NaN
655,Wind Turbine Engineer,Wind - Get Into Energy,61,6,0,1,324.730469,500,750,375000,1,http://getintoenergy.com/wp-content/uploads/20...,good
656,Wind Turbine Engineer,Wind-energy programs yield engineering ...,61,7,0,1,36.197266,920,613,563960,0,https://s.hdnux.com/photos/20/15/22/4248747/3/...,good


In [5]:
data=df.drop(columns=['Picture URL'])

In [6]:
data['Manual Label'].describe()

count      133
unique       2
top       good
freq        91
Name: Manual Label, dtype: object

In [7]:
data['Pic Title'].describe()

count                            658
unique                           593
top       Bureau of Labor Statistics
freq                              17
Name: Pic Title, dtype: object

# Exploratory Analysis

# Data Preprocessing

In [8]:
import nltk
ps = nltk.PorterStemmer()
wn=nltk.WordNetLemmatizer()
nltk.download("popular")
stopwords = nltk.corpus.stopwords.words('english')
import re
import string

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

### Clean up text

In [9]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

data['Job Title'] = data['Job Title'].apply(lambda x: clean_text(x.lower()))
data['Pic Title'] = data['Pic Title'].apply(lambda x: clean_text(x.lower()))
data.head()

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label
0,"[3d, modeler]","[top, 19, 3d, modeling, software, picks, ]",1261,0,0,7,276.763672,1917,2301,4411017,2,NaN
1,"[3d, modeler]","[7, tips, beginner, 3d, modelers, ]",1261,1,0,3,44.343750,340,747,253980,5,NaN
2,"[3d, modeler]","[become, expert, 3d, modeler, ]",1261,2,0,2,43.088867,380,660,250800,8,NaN
3,"[3d, modeler]","[3d, modeling, whats, used]",1261,3,0,2,78.623047,330,750,247500,1,NaN
4,"[3d, modeler]","[shoofping, 3d, modeler, ]",1261,4,0,1,97.517578,1075,1909,2052175,3,NaN


In [10]:
data.columns

Index(['Job Title', 'Pic Title', 'Num Resumes', 'Google Position',
       'Strict Face Count', 'Relaxed Face Count', 'KB size', 'Height', 'Width',
       'Resolution', 'Text Regions', 'Manual Label'],
      dtype='object')

### Stem / Lemmatize text

In [11]:
def stemming(text):
  stemmed=[ps.stem(word) for word in text]
  return stemmed

data['Stem Job Title'] = data['Job Title'].apply(lambda x: stemming(x))
data['Stem Pic Title'] = data['Pic Title'].apply(lambda x: stemming(x))
data

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label,Stem Job Title,Stem Pic Title
0,"[3d, modeler]","[top, 19, 3d, modeling, software, picks, ]",1261,0,0,7,276.763672,1917,2301,4411017,2,NaN,"[3d, model]","[top, 19, 3d, model, softwar, pick, ]"
1,"[3d, modeler]","[7, tips, beginner, 3d, modelers, ]",1261,1,0,3,44.343750,340,747,253980,5,NaN,"[3d, model]","[7, tip, beginn, 3d, model, ]"
2,"[3d, modeler]","[become, expert, 3d, modeler, ]",1261,2,0,2,43.088867,380,660,250800,8,NaN,"[3d, model]","[becom, expert, 3d, model, ]"
3,"[3d, modeler]","[3d, modeling, whats, used]",1261,3,0,2,78.623047,330,750,247500,1,NaN,"[3d, model]","[3d, model, what, use]"
4,"[3d, modeler]","[shoofping, 3d, modeler, ]",1261,4,0,1,97.517578,1075,1909,2052175,3,NaN,"[3d, model]","[shoofp, 3d, model, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,"[wind, turbine, engineer]","[wind, turbine, technician, fastestgrowing, ]",61,4,0,0,89.249023,720,1280,921600,0,good,"[wind, turbin, engin]","[wind, turbin, technician, fastestgrow, ]"
654,"[wind, turbine, engineer]","[become, wind, energy, engineer, ]",61,5,0,0,37.949219,200,300,60000,0,NaN,"[wind, turbin, engin]","[becom, wind, energi, engin, ]"
655,"[wind, turbine, engineer]","[wind, get, energy]",61,6,0,1,324.730469,500,750,375000,1,good,"[wind, turbin, engin]","[wind, get, energi]"
656,"[wind, turbine, engineer]","[windenergy, programs, yield, engineering, ]",61,7,0,1,36.197266,920,613,563960,0,good,"[wind, turbin, engin]","[windenergi, program, yield, engin, ]"


In [12]:
data[data['Manual Label']=='bad']

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label,Stem Job Title,Stem Pic Title
29,[assistant],"[google, assistant, 20, change, ]",347272,2,0,0,56.383789,640,1280,819200,1,bad,[assist],"[googl, assist, 20, chang, ]"
40,"[assistant, curator]","[yale, university, art, gallery, appoints, ]",868,1,1,1,57.474609,576,864,497664,0,bad,"[assist, curat]","[yale, univers, art, galleri, appoint, ]"
48,"[assistant, designer]","[assistant, costume, designer, get, media]",5358,1,1,1,19.559570,210,280,58800,0,bad,"[assist, design]","[assist, costum, design, get, media]"
50,"[assistant, designer]","[assistant, designer]",5358,3,0,3,104.434570,479,638,305602,4,bad,"[assist, design]","[assist, design]"
56,"[benefits, consultant]","[benefits, consultant, ]",3890,0,0,1,52.086914,480,720,345600,0,bad,"[benefit, consult]","[benefit, consult, ]"
57,"[benefits, consultant]","[benefits, consultant, ]",3890,1,0,1,20.233398,200,300,60000,0,bad,"[benefit, consult]","[benefit, consult, ]"
58,"[benefits, consultant]","[surprising, ways, benefits, consultant, ]",3890,3,0,1,18.697266,200,300,60000,0,bad,"[benefit, consult]","[surpris, way, benefit, consult, ]"
59,"[benefits, consultant]","[benefits, consultant, ]",3890,4,0,0,28.780273,720,1280,921600,0,bad,"[benefit, consult]","[benefit, consult, ]"
60,"[benefits, consultant]","[benefits, consultant, ]",3890,5,0,1,14.895508,273,375,102375,0,bad,"[benefit, consult]","[benefit, consult, ]"
62,"[benefits, consultant]","[ask, employee, benefits, consultant, ]",3890,7,0,0,61.863281,398,602,239596,0,bad,"[benefit, consult]","[ask, employe, benefit, consult, ]"


In [13]:
data[data['Manual Label']=='good']

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label,Stem Job Title,Stem Pic Title
21,"[architectural, drafter]","[9, firms, offering, architecture, ]",1824,1,0,2,529.085938,1382,2200,3040400,2,good,"[architectur, drafter]","[9, firm, offer, architectur, ]"
24,"[architectural, drafter]","[2017, architectural, drafter, careercastcom]",1824,4,0,1,28.923828,446,600,267600,0,good,"[architectur, drafter]","[2017, architectur, drafter, careercastcom]"
32,[assistant],"[personal, assistant, apps, ]",347272,7,1,2,390.638672,844,1243,1049092,0,good,[assist],"[person, assist, app, ]"
39,"[assistant, curator]","[assistant, curator, ancient, ]",868,0,1,10,640.045898,1500,2000,3000000,0,good,"[assist, curat]","[assist, curat, ancient, ]"
41,"[assistant, curator]","[assistant, curator, nicole, smythejohnson, ]",868,2,0,2,77.317383,720,1280,921600,0,good,"[assist, curat]","[assist, curat, nicol, smythejohnson, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,"[wind, turbine, engineer]","[wind, turbine, technician, ]",61,3,0,2,142.578125,1200,1200,1440000,0,good,"[wind, turbin, engin]","[wind, turbin, technician, ]"
653,"[wind, turbine, engineer]","[wind, turbine, technician, fastestgrowing, ]",61,4,0,0,89.249023,720,1280,921600,0,good,"[wind, turbin, engin]","[wind, turbin, technician, fastestgrow, ]"
655,"[wind, turbine, engineer]","[wind, get, energy]",61,6,0,1,324.730469,500,750,375000,1,good,"[wind, turbin, engin]","[wind, get, energi]"
656,"[wind, turbine, engineer]","[windenergy, programs, yield, engineering, ]",61,7,0,1,36.197266,920,613,563960,0,good,"[wind, turbin, engin]","[windenergi, program, yield, engin, ]"


In [14]:
arr1 = data['Stem Job Title'].values # first information (common small)
arr2 = data['Stem Pic Title'].values # second information
n = arr1.shape[0] # n = 658
lista = []
divide = []
for i in range(n):
  aux =0
  c1 = list(arr1[i])
  for x in arr1[i]:
    c2 = list(arr2[i])
    if(c2.count(x) != 0):
      aux = aux + 1
  lista.append(aux)
  try:
   divide.append(len(c1)/aux)
  except:
   divide.append(0)
print(lista)
print(divide)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 0, 0, 0, 0, 2, 1, 2, 2, 2, 1, 0, 2, 1, 1, 1, 1, 1, 3, 3, 2, 3, 3, 3, 2, 0, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 1, 2, 2, 2, 2, 4, 4, 2, 2, 4, 1, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 1, 0, 2, 2, 1, 0, 3, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 3, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 1, 2, 0, 2, 2, 2, 2, 0, 0, 3, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 0, 3, 3, 0, 2, 4, 0, 1, 0, 2, 0, 0, 3, 3, 3, 3, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 3, 0, 2, 0, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 1, 1, 0, 0, 1, 1, 1, 2, 3, 1, 3, 3, 3, 2, 3, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 

In [15]:
'''arr1 = data['Stem Job Title'].values # first information (common small)
arr2 = data['Stem Pic Title'].values # second information
n = arr1.shape[0] # n = 658
lista = []
divide = []
for i in range(n):
  aux =0
  c1 = list(arr1[i])
  for x in arr1[i]:
    c2 = list(c2[i])
    t1 = x[:4]
    for t2 in c2:
      if(t2[:4].count(t1) != 0):
        aux = aux + 1
  lista.append(aux)
  try:
   divide.append(len(c1)/aux)
  except:
   divide.append(0)
print(lista)
print(divide)'''

IndexError: ignored

In [ ]:
for x in c2:
  print(x[:4])

In [ ]:
data.head()

career, senior

In [ ]:
'''def lemmatized(text):
  lemmatized= [wn.lemmatize(word) for word in text]
  return lemmatized

data['Lemma Job Title'] = data['Job Title'].apply(lambda x: lemmatized(x))
data['Lemma Pic Title'] = data['Pic Title'].apply(lambda x: lemmatized(x))
data'''

###Data split

In [ ]:
data=data.drop(columns=['Job Title','Pic Title'])

In [ ]:
''' Data (with labels) to test the model '''

labaled= data[data['Manual Label'].notna()]
labaled.head()

In [ ]:
#Drop target feature
X = data.drop(columns=['Manual Label','Stem Job Title','Stem Pic Title','KB size','Num Resumes','Google Position','Width','Text Regions']) 
#X = data.drop(columns=['Manual Label','Stem Job Title','Stem Pic Title']) 
#X['Common']=lista
X['Division']=divide
W = X
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

# Predictive Model

In [ ]:
y_pred = KMeans(n_clusters=2).fit_predict(X)

In [ ]:
len(y_pred)

In [ ]:
# get the elements containing label (we will use it to evaluate the method)
y = data['Manual Label'].replace({'good':0, 'bad':1})
index_no = data['Manual Label'].notna()
y_real = y[index_no].astype(int)
X_eval= X[index_no,:]
y_mod = y_pred[index_no]

In [ ]:
y_real.shape, y_mod.shape

In [ ]:
# Classification Report
print('\n\n', classification_report(y_real, y_mod, target_names=['0-Good', '1-Bad']))
# Confusion Matrix
con_matrix = pd.DataFrame(confusion_matrix(y_real, y_mod), columns=['Predicted Good', 'Predicted Bad'], index=['Actual Good', 'Actual Bad'])
  
plt.figure(figsize=(15,10))
sns.heatmap(data=con_matrix, cmap='cool');
plt.title('Model Confusion Matrix')
plt.show();

In [ ]:
W